In [1]:
from Script.General import *

g = General()


# Register Variable

In [2]:
register = [
    ['kas','2','CurrentYearInstant','Kas dan setara kas'],
    ['jumlah_aset_lancar','2','CurrentYearInstant','Jumlah aset lancar'],
    ['persediaan','2','CurrentYearInstant','Persediaan lancar lainnya'],
    ['piutang_pihak_tiga','2','CurrentYearInstant','Piutang usaha pihak ketiga'],
    ['piutang_relasi','2','CurrentYearInstant','Piutang usaha pihak berelasi'],
    ['jumlah_aset_tidak_lancar','2','CurrentYearInstant','Jumlah aset tidak lancar'],
    ['jumlah_aset','2','CurrentYearInstant','Jumlah aset'],
    ['jumlah_liabilitas_jangka_pendek','2','CurrentYearInstant','Jumlah liabilitas jangka pendek'],
    ['jumlah_liabilitas_jangka_panjang','2','CurrentYearInstant','Jumlah liabilitas jangka panjang'],
    ['jumlah_ekuitas','2','CurrentYearInstant','Jumlah ekuitas'],
    ['jumlah_liabilitas','2','CurrentYearInstant','Jumlah liabilitas'],
    ['jumlah_ekuitas','2','CurrentYearInstant','Jumlah ekuitas'],
    ['pendapatan','3','CurrentYearDuration','Penjualan dan pendapatan usaha'],
    ['cogs','3','CurrentYearDuration','Beban pokok penjualan dan pendapatan'],
    ['laba_bruto','3','CurrentYearDuration','Jumlah laba bruto'],
    ['laba_rugi','3','CurrentYearDuration','Jumlah laba (rugi)'],
    ['laba_rugi_komprehensif','3','CurrentYearDuration','Jumlah laba rugi komprehensif'],
    ['EPS','3','CurrentYearDuration','Laba (rugi) per saham dasar dari operasi yang dilanjutkan'],
]

# Industry Categorization

In [3]:
def list_cat(x):
    folder_list = os.listdir(x['folder_path'])
    value = np.nan
    for fd in folder_list:
        if "1210000.html" in fd:
            value = "General"
        elif "6220000.html" in fd:
            value = "Insurance"
        elif "1220000.html" in fd:
            value = "General - liquidity"
        elif "2210000.html" in fd:
            value = "Property"
        elif "8220000.html" in fd:
            value = "Finance Industry"
        elif "4220000.html" in fd :
            value = "Financial and Shariah"
        elif "3210000.html" in fd :
            value = "Infrastructure Industry"
        elif "2220000.html" in fd :
            value = "Property"
        elif "5220000.html" in fd :
            value = "Securities Industry"
        elif "3220000.html" in fd :
            value = "Infrastructure Industry - liqudity"

    
    return value

In [4]:
folder_list = g.add_folder_columns()
folder_list['category'] = folder_list.apply(lambda x:list_cat(x),axis=1)
folder_list.to_csv("category.csv",index=False)

# Script Extract XBRL

In [5]:
folder_list = g.add_folder_columns()
val_list = pd.DataFrame()
i=0

for row in folder_list.itertuples():
    val={}
    folder_path = row.folder_path
    val['Company'] = row.Company
    val['Tahun_Buku'] = row.Tahun_Buku
    val['Tanggal Rilis'] = row.Date

    for x in register:

        try:
            dire = os.listdir(folder_path)
            for file_name in dire:
                if x[1] in file_name[1]:
                    file_path = file_name
                
            val[f'{x[0]}'] = g.get_data(folder_path,file_path,x[2],x[3])
            
        except:
            pass
    
    val = pd.DataFrame(val,index=[i])
    val_list = pd.concat([val_list,val])
    i+=1



# Script Extract Tanggal

In [6]:
from datetime import datetime, timedelta

In [7]:
month_converter = {
    'Januari' : 'January',
    'Februari' : 'February',
    'Maret' : 'March',
    'April' : 'April',
    'Mei' : 'May',
    'Juni' : 'June',
    'Juli' : 'July',
    'Agustus' : 'August',
    'September' : 'September',
    'Oktober' : 'October',
    'November' : 'November',
    'Desember' : 'December'
}

for indonesia_month, english_month in month_converter.items():
    val_list['Tanggal Rilis'] = val_list['Tanggal Rilis'].str.replace(indonesia_month, english_month)

In [8]:
date_format = "%d %B %Y | %H:%M"
val_list['Tanggal Rilis'] = pd.to_datetime(val_list['Tanggal Rilis'],format=date_format)

In [9]:
val_list['End of Next Month'] = val_list['Tanggal Rilis'] + timedelta(days=14)

In [10]:
val_list

,Company,Tahun_Buku,Tanggal Rilis,kas,jumlah_aset_lancar,jumlah_aset_tidak_lancar,jumlah_aset,jumlah_liabilitas_jangka_pendek,jumlah_liabilitas_jangka_panjang,jumlah_ekuitas,jumlah_liabilitas,laba_bruto,laba_rugi,laba_rugi_komprehensif,EPS,End of Next Month
0,AALI,2020,2021-02-24 14:27:00,"978,892","5,937,890","21,843,341","27,781,231","1,792,506","6,740,931","19,247,794","8,533,437","2,962,891","893,779","467,253",432.84,2021-03-31 14:27:00
1,ABBA,2020,2021-08-25 23:48:00,"26,724,525,401","64,137,591,087","157,511,693,082","221,649,284,169","235,223,369,732","90,136,585,445","103,710,671,008","325,359,955,177","76,544,027,930","58,331,871,976","149,827,890,727",16.3,2021-09-30 23:48:00
2,ABDA,2020,2021-03-31 16:50:00,"652,083,696",NaN,NaN,"2,477,781,648",NaN,NaN,"1,387,236,002","1,090,545,646",NaN,"138,190,287","170,780,025",223,2021-05-31 16:50:00
3,ABMM,2020,2021-05-31 15:01:00,"109,212,396","314,160,618","513,076,561","827,237,179","262,691,486","402,796,549","161,749,144","665,488,035","101,515,013","37,740,064","58,459,398",0.01295,2021-07-31 15:01:00
4,ACES,2020,2021-05-06 17:16:00,"2,219,784,801,023","5,034,737,166,320","2,212,326,727,974","7,247,063,894,294","844,928,054,206","1,179,893,285,690","5,222,242,554,398","2,024,821,339,896","3,659,181,805,722","731,310,571,351","833,806,976,851",42.86,2021-06-30 17:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2243,ZATA,2022,2023-04-03 15:02:00,"24,478,415,707","456,838,923,139","268,853,860,763","725,692,783,902","57,066,818,289","156,686,492,263","511,939,473,350","213,753,310,552","86,739,903,981","5,768,874,815","5,464,071,480",1.34,2023-05-31 15:02:00
2244,ZBRA,2022,2023-04-10 17:54:00,"17,158,493,885","2,446,170,156,290","710,104,210,177","3,156,274,366,467","1,728,423,126,211","167,850,853,357","1,260,000,386,899","1,896,273,979,568","350,862,487,265","112,308,310,596","109,277,572,669",46.45,2023-05-31 17:54:00
2245,ZINC,2022,2023-06-26 21:49:00,"53,464,902,205","677,776,438,628","1,798,177,599,273","2,475,954,037,901","708,012,768,442","994,819,621,627","773,121,647,832","1,702,832,390,069","177,430,748,649","114,709,135,630","114,149,127,220",4.08,2023-07-31 21:49:00
2246,ZONE,2022,2023-04-03 15:09:00,"4,910,325,646","374,779,329,475","277,001,901,483","651,781,230,958","199,174,823,581","105,758,180,179","346,848,227,198","304,933,003,760","376,702,968,499","72,940,513,980","72,344,079,526",84,2023-05-31 15:09:00


# Tarik Data Saham

In [11]:
import yfinance as yf

In [12]:
def avg_seven_day(x):
    company_code = f'{x["Company"]}.JK'
    start_date = x['Tanggal Rilis'].strftime('%Y-%m-%d')
    end_date = x['End of Next Month'].strftime('%Y-%m-%d')

    data = yf.download(company_code,start_date,end_date)
    value = data['Adj Close'].head(7).mean()
    return value

In [13]:
val_list['Avg Harga Saham 7 Hari'] = val_list.apply(lambda x: avg_seven_day(x),axis=1)
val_list.drop(['End of Next Month'],axis=1,inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


1 Failed download:
['HDTX.JK']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******


1 Failed download:
['KETR.JK']: Exception("%ticker%: Data doesn't exist for startDate = 1617642000, endDate = 1622394000")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['TRIO.JK']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******


1 Failed download:
['UNSP.JK']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-04-06 -> 2023-05-31)')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

In [14]:
val_list.to_csv('data_raw.csv',index=False)